In [35]:
import numpy as np
import glob
import cv2

In [48]:
class SVM:
    def __init__(self, learning_rate=1e-3, lambda_param=1e-2, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None

    def _init_weights_bias(self, X):
        n_features = X.shape[1]
        self.w = np.zeros(n_features)
        self.b = 0

    def _get_cls_map(self, y):
        return np.where(y <= 0, -1, 1)

    def _satisfy_constraint(self, x, idx):
        linear_model = np.dot(x, self.w) + self.b 
        return self.cls_map[idx] * linear_model >= 1
    
    def _get_gradients(self, constrain, x, idx):
        if constrain:
            dw = self.lambda_param * self.w
            db = 0
            return dw, db
        
        dw = self.lambda_param * self.w - np.dot(self.cls_map[idx], x)
        db = - self.cls_map[idx]
        return dw, db
    
    def _update_weights_bias(self, dw, db):
        self.w -= self.lr * dw
        self.b -= self.lr * db
    
    def fit(self, X, y):
        self._init_weights_bias(X)
        self.cls_map = self._get_cls_map(y)

        for _ in range(self.n_iters):
            for idx, x in enumerate(X):
                constrain = self._satisfy_constraint(x, idx)
                dw, db = self._get_gradients(constrain, x, idx)
                self._update_weights_bias(dw, db)
    
    def predict(self, X):
        estimate = np.dot(X, self.w) + self.b
        prediction = np.sign(estimate)
        return np.where(prediction == -1, 0, 1)

In [49]:
# 0 = giraffe, 1 = zebra

y = []

for i in range(775):
    y.append(0)
    
for i in range(697):
    y.append(1)

In [50]:
# read images

X = [cv2.imread(file, cv2.IMREAD_GRAYSCALE) for file in glob.glob("/Users/ahmetuten/Desktop/Ders/FIZ437E/images/*.jpg")]
X = np.array(X)

In [51]:
# images to 1d array
raveled = []

for i in range(len(X)):
    raveled.append(X[i].ravel())
    
raveled = np.array(raveled)

In [52]:
# scale the pixel values
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_raveled = scaler.fit_transform(raveled)

In [53]:
perc = 0.9

x_train = x_raveled[:int(np.floor(len(x_raveled)*perc))]
x_test = x_raveled[int(np.floor(len(x_raveled)*perc)):]

y_train = y[:int(np.floor(len(y)*perc))]
y_test = y[int(np.floor(len(y)*perc)):]

In [54]:
y_train = np.array(y_train)

In [55]:
clf = SVM()
clf.fit(x_train, y_train)

In [56]:
y_pred = clf.predict(x_test)
print(y_pred)

[0 0 0 0 1 0 0 0 1 0 1 1 1 1 0 1 0 0 1 1 1 1 1 0 1 0 0 0 0 1 0 1 1 1 1 1 0
 0 0 0 0 1 1 1 0 1 0 1 0 0 1 1 0 0 1 0 1 0 0 0 0 0 0 1 1 0 0 1 1 0 1 0 0 1
 0 0 1 0 0 1 0 1 0 0 1 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 1 1 0 1 0 1
 0 1 0 0 0 0 1 0 0 0 1 1 0 1 0 1 0 1 0 0 0 1 1 0 1 0 0 1 0 0 1 0 0 0 1 1 1]


In [57]:
# accuracy
acc = np.sum(np.equal(y_test, y_pred)) / len(y_test)
print(acc)

0.42567567567567566
